# 1. Sapm Detection

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import metrics
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tf.get_logger().setLevel('ERROR')

In [ ]:
#  https://www.kaggle.com/uciml/sms-spam-collection-dataset
data = pd.read_csv(r'spam.csv', encoding='latin1')
print('sample number:',len(data))

In [ ]:
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
data

In [ ]:
data.drop_duplicates(subset=['v2'], inplace=True)

In [ ]:
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data = data[:200]
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data

# numlist = list(range(len(data)))
# data = data.sample(400, random_state=100)
# numlist = list(range(len(data)))
# data = data.set_index(pd.Index(numlist))
# data

In [ ]:
data['v1'].value_counts()

In [ ]:
# data.drop(data[:380][data['v1'][:380] == 0].index, inplace=True)
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data

In [ ]:
X = data['v2']
y = data['v1']
y = y.astype(float)

In [ ]:
y.value_counts()

In [ ]:
print(len(X), X[0])

In [ ]:
# y = tf.convert_to_tensor(y, dtype=tf.float32, dtype_hint=None, name=None)

In [ ]:
print(len(y), y[0])

In [ ]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-2_H-128_A-2'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

In [ ]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
#     net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)     # activation=None
    return tf.keras.Model(text_input, net)

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

batch = 32
epochs = 10

from keras import optimizers

steps_per_epoch = int(len(X)*0.4)  # length of train data
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
X

In [ ]:
y_hard = pd.DataFrame(y)
y_hard

# 1-0. Generating Prob_labels

In [ ]:
X_gen = tf.convert_to_tensor(X, dtype=tf.string, dtype_hint=None, name=None)
gen_model = build_classifier_model()
opt = optimization.create_optimizer(init_lr=0.0001,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
gen_model.compile(loss='BinaryCrossentropy', optimizer=opt, metrics=['accuracy'])
history = gen_model.fit(X_gen, y_hard, validation_split=0.2, epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
plt.plot(history.history['loss'], label='loss')
plt.ylim([0, 1])
plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
plt.ylabel('Loss',fontweight="bold",fontsize = 15)
plt.title("Cost Function",fontweight="bold",fontsize = 20)
plt.legend()
plt.show()
result = gen_model.predict(X_gen, verbose=0)
prob_label = list(result.reshape(len(X),))
y = pd.DataFrame(prob_label)
y

In [ ]:
#
c = 0.5
y_edge = []
for i in range(len(y)):
    if list(y_hard['v1'])[i] == 0:
        if list(y[0])[i] <= c:
            y_edge.append(0)  # easy sample
        else:
            y_edge.append(2) # hard sample
    if list(y_hard['v1'])[i] == 1:
        if list(y[0])[i] >= 1-c:
            y_edge.append(0)  # easy sample
        else:
            y_edge.append(2) # hard sample
y_edge = pd.DataFrame(y_edge)
y_edge.value_counts()   

In [ ]:
#
edge_list = list(y_edge[y_edge[0] == 2].index)
normal_list = list(y_edge[y_edge[0] == 0].index)
print(edge_list)

In [ ]:
r = list(y_edge[0]).count(0)/list(y_edge[0]).count(2)   # normal/edge
alpha = (r-1)/(2*r)
print(r, alpha)

# 1-1. Focal(Hard) and SLS(Hard/alpha)

In [ ]:
#
r = list(y_edge[0]).count(0)/list(y_edge[0]).count(2)   # normal/edge
alpha = (r-1)/(2*r)
B = [0.00, alpha]

for t in range(10):    # 10 times repeat      
    res = pd.DataFrame({'Focal':[0, 0, 0]}, index = ['Total','Edge','Normal']) 
    # Focal
    print('#'*50,'Focal','#'*50)
    list_total = []
    list_edge = []
    list_normal = []  
    focal_model = build_classifier_model()   

    n_iter = 0
    for train_index, test_index in skf.split(X, y_edge):  # straticiation by y_edge
        n_iter += 1
        X_train = X[train_index]
        y_train= y_hard.iloc[train_index]     # train with hard labels
        if n_iter == 1:
            print(y_train.value_counts())
        X_test = X[test_index]
        y_test= y_hard.iloc[test_index]     # test with hard labels
        test_edge_list = []
        for index in edge_list:
            if index in test_index:
                test_edge_list.append(index)
        X_test_edge = X[test_edge_list]
        y_test_edge = y_hard.iloc[test_edge_list]     # test with hard labels
        test_normal_list = []
        for index in normal_list:
            if index in test_index:
                test_normal_list.append(index)
        X_test_normal = X[test_normal_list]
        y_test_normal = y_hard.iloc[test_normal_list]     # test with hard labels
        
        X_train = tf.convert_to_tensor(X_train, dtype=tf.string, dtype_hint=None, name=None)
        X_test = tf.convert_to_tensor(X_test, dtype=tf.string, dtype_hint=None, name=None)
        X_test_edge = tf.convert_to_tensor(X_test_edge, dtype=tf.string, dtype_hint=None, name=None)
        X_test_normal = tf.convert_to_tensor(X_test_normal, dtype=tf.string, dtype_hint=None, name=None)

        opt = optimization.create_optimizer(init_lr=0.0001,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
        focal_model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy'])
        history = focal_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#         plt.plot(history.history['loss'], label='loss')
#         plt.ylim([0, 1])
#         plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#         plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#         plt.title("Cost Function",fontweight="bold",fontsize = 20)
#         plt.legend()
#         plt.show()

        # TEST (total)
        predicted_total = np.round(focal_model.predict(X_test, verbose=0))
        list_total.append(metrics.accuracy_score(y_test, predicted_total))
        # TEST (edge)
        predicted_edge = np.round(focal_model.predict(X_test_edge, verbose=0))
        list_edge.append(metrics.accuracy_score(y_test_edge, predicted_edge))
        # TEST (normal)
        predicted_normal = np.round(focal_model.predict(X_test_normal, verbose=0))
        list_normal.append(metrics.accuracy_score(y_test_normal, predicted_normal))
            
    res['Focal'] = [np.mean(list_total), np.mean(list_edge), np.mean(list_normal)]
    print([np.mean(list_total), np.mean(list_edge), np.mean(list_normal)])

    for b in B:
        print('#'*50,'SLS',b,'#'*50)
        y_sls = []
        for i in range(len(y_hard)):
            if list(y_hard['v1'])[i] == 0:
                if prob_label[i] <= c:
                    y_sls.append(b)  # easy sample
                else:
                    y_sls.append(0) # (or 0-b) hard sample
            if list(y_hard['v1'])[i] == 1:
                if prob_label[i] >= 1-c:
                    y_sls.append(1-b)  # easy sample
                else:
                    y_sls.append(1) # (or 1+b) hard sample
        y_sls = pd.DataFrame(y_sls)     

        sls_total = []
        sls_edge = []
        sls_normal = []
        model_sls = build_classifier_model() 

        n_iter = 0
        for train_index, test_index in skf.split(X, y_edge):  # straticiation by y_edge
            n_iter += 1
            X_train = X[train_index]
            y_sls_train = y_sls.iloc[train_index]     # train with sls labels
            if n_iter == 1:
                print(y_sls_train.value_counts())
            X_test = X[test_index]
            y_test= y_hard.iloc[test_index]     # test with hard labels
            test_edge_list = []
            for index in edge_list:
                if index in test_index:
                    test_edge_list.append(index)
            X_test_edge = X[test_edge_list]
            y_test_edge = y_hard.iloc[test_edge_list]     # test with hard labels
            test_normal_list = []
            for index in normal_list:
                if index in test_index:
                    test_normal_list.append(index)
            X_test_normal = X[test_normal_list]
            y_test_normal = y_hard.iloc[test_normal_list]     # test with hard labels

            X_train = tf.convert_to_tensor(X_train, dtype=tf.string, dtype_hint=None, name=None)
            X_test = tf.convert_to_tensor(X_test, dtype=tf.string, dtype_hint=None, name=None)
            X_test_edge = tf.convert_to_tensor(X_test_edge, dtype=tf.string, dtype_hint=None, name=None)
            X_test_normal = tf.convert_to_tensor(X_test_normal, dtype=tf.string, dtype_hint=None, name=None)
            
            # MLP_BCE(y_005)
            opt = optimization.create_optimizer(init_lr=0.0001,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
            model_sls.compile(loss='BinaryCrossentropy', optimizer=opt, metrics=['accuracy'])
            history = model_sls.fit(X_train, y_sls_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#             plt.plot(history.history['loss'], label='loss')
#             plt.ylim([0, 1])
#             plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#             plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#             plt.title("Cost Function",fontweight="bold",fontsize = 20)
#             plt.legend()
#             plt.show()
            
            # TEST (total)
            predicted_total = np.round(model_sls.predict(X_test, verbose=0))
            sls_total.append(metrics.accuracy_score(y_test, predicted_total))
            # TEST (edge)
            predicted_edge = np.round(model_sls.predict(X_test_edge, verbose=0))
            sls_edge.append(metrics.accuracy_score(y_test_edge, predicted_edge))
            # TEST (normal)
            predicted_normal = np.round(model_sls.predict(X_test_normal, verbose=0))
            sls_normal.append(metrics.accuracy_score(y_test_normal, predicted_normal))
                       
        res['SLS({})'.format(b)] = [np.mean(sls_total), np.mean(sls_edge), np.mean(sls_normal)]
        print([np.mean(sls_total), np.mean(sls_edge), np.mean(sls_normal)])          
    res.to_csv("BERT_SPAM_5CV(SLS_c0.5)_alphaimproved.csv", mode = 'a', float_format='%.4g')

# 2. Reuters News

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import metrics
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tf.get_logger().setLevel('ERROR')

In [ ]:
data = pd.read_csv("ModHayes_train.csv")
data

In [ ]:
data = data[["text","topics"]]

In [ ]:
data['topics'].value_counts()[:10]

In [ ]:
isblank = data['topics'] == "[]"
isacq = data['topics'] == "['acq']"
data = data[isblank | isacq]
data = data.dropna()
data

In [ ]:
data['topics'] = data['topics'].replace(["[]","['acq']"],[0,1])
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data = data[:350]
data

In [ ]:
data['topics'].value_counts()

In [ ]:
data.drop(data[:295][data['topics'][:295] == 0].index, inplace=True)
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data

In [ ]:
data['topics'].value_counts()

In [ ]:
X = data['text']
y = data['topics']
y = y.astype(float)

In [ ]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-2_H-128_A-2'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

In [ ]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
#     net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)     # activation=None
    return tf.keras.Model(text_input, net)

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

batch = 64
epochs = 10

from keras import optimizers

steps_per_epoch = int(len(X)*0.4)  # length of train data
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
X

In [ ]:
y_hard = pd.DataFrame(y)
y_hard

# 2-0. Generating Prob_labels

In [ ]:
X_gen = tf.convert_to_tensor(X, dtype=tf.string, dtype_hint=None, name=None)
gen_model = build_classifier_model()
opt = optimization.create_optimizer(init_lr=0.0002,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
gen_model.compile(loss='BinaryCrossentropy', optimizer=opt, metrics=['accuracy'])
history = gen_model.fit(X_gen, y_hard, validation_split=0.2, epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
plt.plot(history.history['loss'], label='loss')
plt.ylim([0, 1])
plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
plt.ylabel('Loss',fontweight="bold",fontsize = 15)
plt.title("Cost Function",fontweight="bold",fontsize = 20)
plt.legend()
plt.show()
result = gen_model.predict(X_gen, verbose=0)
prob_label = list(result.reshape(len(X),))
y = pd.DataFrame(prob_label)
y

In [ ]:
#
c = 0.5
y_edge = []
for i in range(len(y)):
    if list(y_hard['topics'])[i] == 0:
        if list(y[0])[i] <= c:
            y_edge.append(0)  # easy sample
        else:
            y_edge.append(2) # hard sample
    if list(y_hard['topics'])[i] == 1:
        if list(y[0])[i] >= 1-c:
            y_edge.append(0)  # easy sample
        else:
            y_edge.append(2) # hard sample
y_edge = pd.DataFrame(y_edge)
y_edge.value_counts()   

In [ ]:
#
edge_list = list(y_edge[y_edge[0] == 2].index)
normal_list = list(y_edge[y_edge[0] == 0].index)
print(edge_list)

In [ ]:
r = list(y_edge[0]).count(0)/list(y_edge[0]).count(2)   # normal/edge
alpha = (r-1)/(2*r)
print(r, alpha)

# 2-1. Focal(Hard) and SLS(Hard/alpha)

In [ ]:
#
r = list(y_edge[0]).count(0)/list(y_edge[0]).count(2)   # normal/edge
alpha = (r-1)/(2*r)
B = [0.00, alpha]

for t in range(10):    # 10 times repeat      
    res = pd.DataFrame({'Focal':[0, 0, 0]}, index = ['Total','Edge','Normal']) 
    # Focal
    print('#'*50,'Focal','#'*50)
    list_total = []
    list_edge = []
    list_normal = []  
    focal_model = build_classifier_model()   

    n_iter = 0
    for train_index, test_index in skf.split(X, y_edge):  # straticiation by y_edge
        n_iter += 1
        X_train = X[train_index]
        y_train= y_hard.iloc[train_index]     # train with hard labels
        if n_iter == 1:
            print(y_train.value_counts())
        X_test = X[test_index]
        y_test= y_hard.iloc[test_index]     # test with hard labels
        test_edge_list = []
        for index in edge_list:
            if index in test_index:
                test_edge_list.append(index)
        X_test_edge = X[test_edge_list]
        y_test_edge = y_hard.iloc[test_edge_list]     # test with hard labels
        test_normal_list = []
        for index in normal_list:
            if index in test_index:
                test_normal_list.append(index)
        X_test_normal = X[test_normal_list]
        y_test_normal = y_hard.iloc[test_normal_list]     # test with hard labels
        
        X_train = tf.convert_to_tensor(X_train, dtype=tf.string, dtype_hint=None, name=None)
        X_test = tf.convert_to_tensor(X_test, dtype=tf.string, dtype_hint=None, name=None)
        X_test_edge = tf.convert_to_tensor(X_test_edge, dtype=tf.string, dtype_hint=None, name=None)
        X_test_normal = tf.convert_to_tensor(X_test_normal, dtype=tf.string, dtype_hint=None, name=None)

        opt = optimization.create_optimizer(init_lr=0.0002,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
        focal_model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy'])
        history = focal_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#         plt.plot(history.history['loss'], label='loss')
#         plt.ylim([0, 1])
#         plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#         plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#         plt.title("Cost Function",fontweight="bold",fontsize = 20)
#         plt.legend()
#         plt.show()

        # TEST (total)
        predicted_total = np.round(focal_model.predict(X_test, verbose=0))
        list_total.append(metrics.accuracy_score(y_test, predicted_total))
        # TEST (edge)
        predicted_edge = np.round(focal_model.predict(X_test_edge, verbose=0))
        list_edge.append(metrics.accuracy_score(y_test_edge, predicted_edge))
        # TEST (normal)
        predicted_normal = np.round(focal_model.predict(X_test_normal, verbose=0))
        list_normal.append(metrics.accuracy_score(y_test_normal, predicted_normal))
            
    res['Focal'] = [np.mean(list_total), np.mean(list_edge), np.mean(list_normal)]
    print([np.mean(list_total), np.mean(list_edge), np.mean(list_normal)])

    for b in B:
        print('#'*50,'SLS',b,'#'*50)
        y_sls = []
        for i in range(len(y_hard)):
            if list(y_hard['topics'])[i] == 0:
                if prob_label[i] <= c:
                    y_sls.append(b)  # easy sample
                else:
                    y_sls.append(0) # (or 0-b) hard sample
            if list(y_hard['topics'])[i] == 1:
                if prob_label[i] >= 1-c:
                    y_sls.append(1-b)  # easy sample
                else:
                    y_sls.append(1) # (or 1+b) hard sample
        y_sls = pd.DataFrame(y_sls)     

        sls_total = []
        sls_edge = []
        sls_normal = []
        model_sls = build_classifier_model() 

        n_iter = 0
        for train_index, test_index in skf.split(X, y_edge):  # straticiation by y_edge
            n_iter += 1
            X_train = X[train_index]
            y_sls_train = y_sls.iloc[train_index]     # train with sls labels
            if n_iter == 1:
                print(y_sls_train.value_counts())
            X_test = X[test_index]
            y_test= y_hard.iloc[test_index]     # test with hard labels
            test_edge_list = []
            for index in edge_list:
                if index in test_index:
                    test_edge_list.append(index)
            X_test_edge = X[test_edge_list]
            y_test_edge = y_hard.iloc[test_edge_list]     # test with hard labels
            test_normal_list = []
            for index in normal_list:
                if index in test_index:
                    test_normal_list.append(index)
            X_test_normal = X[test_normal_list]
            y_test_normal = y_hard.iloc[test_normal_list]     # test with hard labels

            X_train = tf.convert_to_tensor(X_train, dtype=tf.string, dtype_hint=None, name=None)
            X_test = tf.convert_to_tensor(X_test, dtype=tf.string, dtype_hint=None, name=None)
            X_test_edge = tf.convert_to_tensor(X_test_edge, dtype=tf.string, dtype_hint=None, name=None)
            X_test_normal = tf.convert_to_tensor(X_test_normal, dtype=tf.string, dtype_hint=None, name=None)
            
            # MLP_BCE(y_005)
            opt = optimization.create_optimizer(init_lr=0.0002,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
            model_sls.compile(loss='BinaryCrossentropy', optimizer=opt, metrics=['accuracy'])
            history = model_sls.fit(X_train, y_sls_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#             plt.plot(history.history['loss'], label='loss')
#             plt.ylim([0, 1])
#             plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#             plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#             plt.title("Cost Function",fontweight="bold",fontsize = 20)
#             plt.legend()
#             plt.show()
            
            # TEST (total)
            predicted_total = np.round(model_sls.predict(X_test, verbose=0))
            sls_total.append(metrics.accuracy_score(y_test, predicted_total))
            # TEST (edge)
            predicted_edge = np.round(model_sls.predict(X_test_edge, verbose=0))
            sls_edge.append(metrics.accuracy_score(y_test_edge, predicted_edge))
            # TEST (normal)
            predicted_normal = np.round(model_sls.predict(X_test_normal, verbose=0))
            sls_normal.append(metrics.accuracy_score(y_test_normal, predicted_normal))
                       
        res['SLS({})'.format(b)] = [np.mean(sls_total), np.mean(sls_edge), np.mean(sls_normal)]
        print([np.mean(sls_total), np.mean(sls_edge), np.mean(sls_normal)])          
    res.to_csv("BERT_RNEWS_5CV(SLS_c0.5)_alphaimproved.csv", mode = 'a', float_format='%.4g')

# 3. IMDB Sentiment Analysis

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tf.get_logger().setLevel('ERROR')

In [ ]:
data = pd.read_csv(r"IMDB Dataset.csv")
data

In [ ]:
data['sentiment'] = data['sentiment'].replace(['negative','positive'],[0,1])
data

In [ ]:
X_train = data['review'][:150]
y_train = data['sentiment'][:150]

In [ ]:
print(len(X_train), len(y_train))

In [ ]:
pd.Series(y_train).value_counts()

In [ ]:
# Making lists of index
idx_0 = []
idx_1 = []
for i in range(len(y_train)):
    if list(y_train)[i] == 0:
        idx_0.append(i)
    if list(y_train)[i] == 1:
        idx_1.append(i)
print(len(idx_0), len(idx_1))

In [ ]:
idx = idx_0[:50] + idx_1[:50]
idx.sort()
print(len(idx), idx)

In [ ]:
X = []
y = []
for i in idx:
    X.append(X_train[i])
    y.append(y_train[i])
X = np.array(X)
y = np.array(y)
y = y.astype(float)
print(X.shape, y.shape)

In [ ]:
print(pd.Series(y).value_counts())

In [ ]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-2_H-128_A-2'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

In [ ]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
#     net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)     # activation=None
    return tf.keras.Model(text_input, net)

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

batch = 32
epochs = 10

from keras import optimizers

steps_per_epoch = int(len(X)*0.4)  # length of train data
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
X = pd.Series(X)
X

In [ ]:
y_hard = pd.DataFrame(y)
y_hard

# 3-0. Generating Prob_labels

In [ ]:
X_gen = tf.convert_to_tensor(X, dtype=tf.string, dtype_hint=None, name=None)
gen_model = build_classifier_model()
opt = optimization.create_optimizer(init_lr=0.0002,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
gen_model.compile(loss='BinaryCrossentropy', optimizer=opt, metrics=['accuracy'])
history = gen_model.fit(X_gen, y_hard, validation_split=0.2, epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
plt.plot(history.history['loss'], label='loss')
plt.ylim([0, 1])
plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
plt.ylabel('Loss',fontweight="bold",fontsize = 15)
plt.title("Cost Function",fontweight="bold",fontsize = 20)
plt.legend()
plt.show()
result = gen_model.predict(X_gen, verbose=0)
prob_label = list(result.reshape(len(X),))
y = pd.DataFrame(prob_label)
y

In [ ]:
#
c = 0.5
y_edge = []
for i in range(len(y)):
    if list(y_hard[0])[i] == 0:
        if list(y[0])[i] <= c:
            y_edge.append(0)  # easy sample
        else:
            y_edge.append(2) # hard sample
    if list(y_hard[0])[i] == 1:
        if list(y[0])[i] >= 1-c:
            y_edge.append(0)  # easy sample
        else:
            y_edge.append(2) # hard sample
y_edge = pd.DataFrame(y_edge)
y_edge.value_counts()   

In [ ]:
#
edge_list = list(y_edge[y_edge[0] == 2].index)
normal_list = list(y_edge[y_edge[0] == 0].index)
print(edge_list)

In [ ]:
r = list(y_edge[0]).count(0)/list(y_edge[0]).count(2)   # normal/edge
alpha = (r-1)/(2*r)
print(r, alpha)

# 3-1. Focal(Hard) and SLS(Hard/alpha)

In [ ]:
#
r = list(y_edge[0]).count(0)/list(y_edge[0]).count(2)   # normal/edge
alpha = (r-1)/(2*r)
B = [0.00, alpha]

for t in range(10):    # 10 times repeat      
    res = pd.DataFrame({'Focal':[0, 0, 0]}, index = ['Total','Edge','Normal']) 
    # Focal
    print('#'*50,'Focal','#'*50)
    list_total = []
    list_edge = []
    list_normal = []  
    focal_model = build_classifier_model()   

    n_iter = 0
    for train_index, test_index in skf.split(X, y_edge):  # straticiation by y_edge
        n_iter += 1
        X_train = X[train_index]
        y_train= y_hard.iloc[train_index]     # train with hard labels
        if n_iter == 1:
            print(y_train.value_counts())
        X_test = X[test_index]
        y_test= y_hard.iloc[test_index]     # test with hard labels
        test_edge_list = []
        for index in edge_list:
            if index in test_index:
                test_edge_list.append(index)
        X_test_edge = X[test_edge_list]
        y_test_edge = y_hard.iloc[test_edge_list]     # test with hard labels
        test_normal_list = []
        for index in normal_list:
            if index in test_index:
                test_normal_list.append(index)
        X_test_normal = X[test_normal_list]
        y_test_normal = y_hard.iloc[test_normal_list]     # test with hard labels
        
        X_train = tf.convert_to_tensor(X_train, dtype=tf.string, dtype_hint=None, name=None)
        X_test = tf.convert_to_tensor(X_test, dtype=tf.string, dtype_hint=None, name=None)
        X_test_edge = tf.convert_to_tensor(X_test_edge, dtype=tf.string, dtype_hint=None, name=None)
        X_test_normal = tf.convert_to_tensor(X_test_normal, dtype=tf.string, dtype_hint=None, name=None)

        opt = optimization.create_optimizer(init_lr=0.0002,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
        focal_model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy'])
        history = focal_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#         plt.plot(history.history['loss'], label='loss')
#         plt.ylim([0, 1])
#         plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#         plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#         plt.title("Cost Function",fontweight="bold",fontsize = 20)
#         plt.legend()
#         plt.show()

        # TEST (total)
        predicted_total = np.round(focal_model.predict(X_test, verbose=0))
        list_total.append(metrics.accuracy_score(y_test, predicted_total))
        # TEST (edge)
        predicted_edge = np.round(focal_model.predict(X_test_edge, verbose=0))
        list_edge.append(metrics.accuracy_score(y_test_edge, predicted_edge))
        # TEST (normal)
        predicted_normal = np.round(focal_model.predict(X_test_normal, verbose=0))
        list_normal.append(metrics.accuracy_score(y_test_normal, predicted_normal))
            
    res['Focal'] = [np.mean(list_total), np.mean(list_edge), np.mean(list_normal)]
    print([np.mean(list_total), np.mean(list_edge), np.mean(list_normal)])

    for b in B:
        print('#'*50,'SLS',b,'#'*50)
        y_sls = []
        for i in range(len(y_hard)):
            if list(y_hard[0])[i] == 0:
                if prob_label[i] <= c:
                    y_sls.append(b)  # easy sample
                else:
                    y_sls.append(0) # (or 0-b) hard sample
            if list(y_hard[0])[i] == 1:
                if prob_label[i] >= 1-c:
                    y_sls.append(1-b)  # easy sample
                else:
                    y_sls.append(1) # (or 1+b) hard sample
        y_sls = pd.DataFrame(y_sls)     

        sls_total = []
        sls_edge = []
        sls_normal = []
        model_sls = build_classifier_model() 

        n_iter = 0
        for train_index, test_index in skf.split(X, y_edge):  # straticiation by y_edge
            n_iter += 1
            X_train = X[train_index]
            y_sls_train = y_sls.iloc[train_index]     # train with sls labels
            if n_iter == 1:
                print(y_sls_train.value_counts())
            X_test = X[test_index]
            y_test= y_hard.iloc[test_index]     # test with hard labels
            test_edge_list = []
            for index in edge_list:
                if index in test_index:
                    test_edge_list.append(index)
            X_test_edge = X[test_edge_list]
            y_test_edge = y_hard.iloc[test_edge_list]     # test with hard labels
            test_normal_list = []
            for index in normal_list:
                if index in test_index:
                    test_normal_list.append(index)
            X_test_normal = X[test_normal_list]
            y_test_normal = y_hard.iloc[test_normal_list]     # test with hard labels

            X_train = tf.convert_to_tensor(X_train, dtype=tf.string, dtype_hint=None, name=None)
            X_test = tf.convert_to_tensor(X_test, dtype=tf.string, dtype_hint=None, name=None)
            X_test_edge = tf.convert_to_tensor(X_test_edge, dtype=tf.string, dtype_hint=None, name=None)
            X_test_normal = tf.convert_to_tensor(X_test_normal, dtype=tf.string, dtype_hint=None, name=None)
            
            # MLP_BCE(y_005)
            opt = optimization.create_optimizer(init_lr=0.0002,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
            model_sls.compile(loss='BinaryCrossentropy', optimizer=opt, metrics=['accuracy'])
            history = model_sls.fit(X_train, y_sls_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#             plt.plot(history.history['loss'], label='loss')
#             plt.ylim([0, 1])
#             plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#             plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#             plt.title("Cost Function",fontweight="bold",fontsize = 20)
#             plt.legend()
#             plt.show()
            
            # TEST (total)
            predicted_total = np.round(model_sls.predict(X_test, verbose=0))
            sls_total.append(metrics.accuracy_score(y_test, predicted_total))
            # TEST (edge)
            predicted_edge = np.round(model_sls.predict(X_test_edge, verbose=0))
            sls_edge.append(metrics.accuracy_score(y_test_edge, predicted_edge))
            # TEST (normal)
            predicted_normal = np.round(model_sls.predict(X_test_normal, verbose=0))
            sls_normal.append(metrics.accuracy_score(y_test_normal, predicted_normal))
                       
        res['SLS({})'.format(b)] = [np.mean(sls_total), np.mean(sls_edge), np.mean(sls_normal)]
        print([np.mean(sls_total), np.mean(sls_edge), np.mean(sls_normal)])          
    res.to_csv("BERT_IMDB_5CV(SLS_c0.5)_alphaimproved.csv", mode = 'a', float_format='%.4g')